## Getting data from the URL

In [1]:
#getting the data from the url
import pandas as pd
df=pd.read_html(r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [2]:
type(df)

list

In [4]:
df=df[0]

In [5]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [34]:
df.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

## Removing cells where Borough is not assigned

In [9]:
df=df[df.Borough != "Not assigned"]

In [10]:
df["Borough"].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [15]:
df=df.reset_index()

In [16]:
df.head()

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
df=df.drop(["index"],axis=1)

In [19]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Checking for redundant values

In [25]:

df["Postal Code"].value_counts(ascending = False)


M7A    1
M2M    1
M6H    1
M6C    1
M1N    1
      ..
M9N    1
M4E    1
M4G    1
M6B    1
M6J    1
Name: Postal Code, Length: 103, dtype: int64

In [26]:
df[df["Postal Code"]=="M5A"]

,Postal Code,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Dealing with Neighbourhood

In [38]:
df[df["Neighbourhood"]=="Not assigned"]

,Postal Code,Borough,Neighbourhood


### Therefore there is no value with missing neighbourhood

In [39]:
df.shape

(103, 3)